In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## General Stuff
################################################################################
from timeUtils import clock, elapsed
from listUtils import getFlatList
from time import sleep
from pandas import DataFrame, Series
from ioUtils import getFile, saveFile
from searchUtils import findDirs, findNearest, findAll
from fileUtils import getDirBasics, getBaseFilename
from unicodedata import normalize
from fsUtils import moveDir, setDir, mkDir, isDir, removeDir
from pandasUtils import getRowDataByColValue, getRowData
import operator
from glob import glob
from os.path import join
from collections import Counter


################################################################################
## Mp3 Stuff
################################################################################
from mp3id import mp3ID


################################################################################
## Database Stuff
################################################################################
from dbBase import dbBase
from mainDB import mainDB
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import masterdb


################################################################################
## Music Stuff
################################################################################
from musicUtils import *
from myMusicDBMap import myMusicDBMap
from musicBase import myMusicBase
from matchMyMusic import matchMyMusic
from matchMusicName import myMusicName
from mergeDB import searchForMutualDBEntries, searchForMutualArtistDBEntries


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-08-02 22:46:13.371017


In [2]:
mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()
#mdb.getKnownDBData()
_, _ = clock("Last Run")

=========================   discogs   =========================
Summary Statistics For DB: Discogs
    Using Known Artists: False
    Found 749471 ID -> Name entries
    Found 684739 Name -> ID entries
    Found 0 Albums
=========================   allmusic   =========================
Summary Statistics For DB: AllMusic
    Using Known Artists: False
    Found 49583 ID -> Name entries
    Found 46068 Name -> ID entries
    Found 213595 Albums
=========================   musicbrainz   =========================
Summary Statistics For DB: MusicBrainz
    Using Known Artists: False
    Found 124197 ID -> Name entries
    Found 105072 Name -> ID entries
    Found 0 Albums
=========================   acebootlegs   =========================
Summary Statistics For DB: AceBootlegs
    Using Known Artists: False
    Found 341 ID -> Name entries
    Found 341 Name -> ID entries
    Found 2838 Albums
=========================   rateyourmusic   =========================
Summary Statistics For DB: R

In [9]:
class dbArtistAlbums:
    def __init__(self, mdb, artistName, debug=False):
        self.mdb               = mdb
        self.debug             = False
        self.artistName        = artistName
        self.knownDBs          = None
        self.dbsToMatch        = None
        self.artistAlbums      = None
        
        self.setArtistAlbums()

        
    def getKnownAlbums(self):
        return self.knownArtistAlbums
        
        
    def setArtistAlbums(self):
        dbMatches    = self.mdb.getArtistDataIDs(self.artistName)
        self.knownDBs     = []
        self.dbsToMatch   = []
        self.artistAlbums = []
        for db,artistID in dbMatches.items():
            #print(db,artistID)
            if artistID is not None:
                dbArtistAlbums = self.mdb.getArtistAlbumsFromID(db, artistID, flatten=True)
                #print("\t",dbArtistAlbums)
                self.artistAlbums.append(dbArtistAlbums)
                self.knownDBs.append(db)
            else:
                self.dbsToMatch.append(db)
        self.knownArtistAlbums = sorted(list(set(getFlatList(self.artistAlbums))))
        print("Found {0} albums for artist {1}".format(len(self.knownArtistAlbums), self.artistName))
        
        
    def findDBMatch(self, cutoff=0.8):
        for db in self.dbsToMatch:
            artistDBIDs = self.mdb.getArtistDBIDs(self.artistName, db, cutoff=cutoff, debug=False)
            for artistMatch,artistMatchIDs in artistDBIDs.items():
                for artistMatchID in artistMatchIDs:
                    artistMatchAlbums = self.mdb.getArtistAlbumsFromID(db, artistMatchID, flatten=True)
                    
                    ma = matchAlbums(cutoff=cutoff)
                    ma.match(artistMatchAlbums, self.knownArtistAlbums)

                    print(ma.maxval)
                    if ma.maxval < cutoff:
                        continue
                    if ma.maxval < bestMatchVal["Ratio"]:
                        continue

                    bestMatch = ma.getBestMatch(myFormattedAlbum
                    bestMatchVal = {"Ratio": ma.maxval, "Dir": dirval, 
                                    "Album": {"Name": bestMatch["Name"],
                                              "Code": bestMatch["Code"],
                                              "MediaType": mediaType}}
                    matchedAlbums[myAlbumName] = bestMatchVal

In [15]:
dbAA = dbArtistAlbums(mdb, "Dr")
#dbAA.getKnownAlbums()
dbAA.findDBMatch(cutoff=0.5)

Found 0 albums for artist Dr


In [62]:

            bestMatchVal = {"Ratio": ratioCut, "Dir": None, "Album": None}

            for mediaType, mediaTypeAlbums in artistDBAlbumsFromID.items():
                if albumType is not None:
                    if mediaType not in self.mdb.getDBAlbumTypeNames(db, albumType):
                        continue

                if self.debug:
                    print("\tMy album: {0}".format(myAlbumName))
                myFormattedAlbum = self.mmn.formatAlbum(myAlbumName, mediaType)

                ma = matchAlbums(cutoff=ratioCut)
                ma.match([myFormattedAlbum], mediaTypeAlbums)

                if ma.maxval < ratioCut or ma.maxval > ratioCut+maxCut:
                    continue
                if ma.maxval < bestMatchVal["Ratio"]:
                    continue

                bestMatch = ma.getBestMatch(myFormattedAlbum)

                bestMatchVal = {"Ratio": ma.maxval, "Dir": dirval, 
                                "Album": {"Name": bestMatch["Name"],
                                          "Code": bestMatch["Code"],
                                          "MediaType": mediaType}}
                matchedAlbums[myAlbumName] = bestMatchVal
                #print("{0: <30}{1: <15}{2: <30} --> {3}".format(artistName, db, myAlbumName, bestMatchVal["Album"]))
                #bestMatchVal["Match"].show(debug=True)

NameError: name 'ratioCut' is not defined

In [23]:
dbAA.setArtistAlbums("
dbAA.aac.knownArtistAlbums

['06/04/1996 - Verizon Wireless Amphitheatre - Virgina Beach, VA',
 "07/24/1999 - Woodstock '99 - Rome, NY",
 '1992-12-21: DMBLive: The Bayou, Washington, DC',
 '1993-01-27: DMBLive: The Flood Zone, Richmond, VA, USA',
 '1993-04-10: DMBLive: The Bayou, Washington, DC, USA',
 '1993-08-19: DMB Live Trax, Volume 20: Wetlands Preserve, New York, NY, USA',
 '1994-03-26: DMBLive: Irving Plaza, New York, NY',
 '1994-04-26: DMBLive: Town Point Park, Norfolk, VA',
 '1994-10-22: DMBLive: Blue Note, Columbia, MO',
 '1994-11-26: DMBLive: Backstage Theatre, Seattle, WA, USA',
 '1995-02-25: DMBLive: Lafayette College, Easton, PA',
 '1995-05-05: DMB Live Trax, Volume 12: L.B. Day Amphitheater, Salem, OR, USA',
 '1995-05-10: DMBLive: Warfield Theatre, San Francisco, CA',
 '1995-07-20: DMBLive: Mud Island Amphitheatre, Memphis, TN',
 '1995-08-23: DMB Live Trax, Volume 5: Meadow Brook Music Festival, Rochester Hills, MI, USA',
 '1996-04-30: DMB Live Trax, Volume 4: Classic Amphitheatre, Richmond, VA, US

In [7]:
from listUtils import getFlatList    

dbs        = mdb.getDBs()

for artistName in mdb.getArtists():
    break

"Weird Al" Yankovic ['AceBootlegs', 'CDandLP', 'DatPiff', 'LastFM', 'MetalStorm', 'MusicStack', 'RateYourMusic', 'RockCorner']


In [ ]:
from matchAlbums import matchAlbums

def searchForArtistDBEntries(mdb, artistName):
    mmb.getArtistAlbumsByArtist("Sweet").getUnmatched()
    albums = 


def searchForArtistAlbumsDBEntries(mdb, artistName, albums, dbsToMatch=None, cutoff=0.7, num=10, debug=False):
    retval     = {}
    

    ######################################################################
    #### Set Known Albums
    ######################################################################
    knownArtistAlbums = albums
    
    
    ######################################################################
    #### Loop Over Missing DBs
    ######################################################################
    if dbsToMatch is None:
        dbsToMatch = mdb.getDBs()
    for db in dbsToMatch:        
        artistDBIDs = mdb.getArtistDBIDs(artistName, db, cutoff=cutoff, num=num, debug=num)
        print("{0: <20}".format(db), end="\t")
        if "Full" in debug:
            print("Found {0} possible artists in DB".format(len(artistDBIDs)))
        else:
            print("")
        
        
        ######################################################################
        #### Search For Matches in Possible IDs
        ######################################################################
        resultD = {"ID": None, "Matches": num, "Score": 0.0, "Best": None}
        for dbArtistName, dbArtistIDs in artistDBIDs.items():
            for dbArtistID in dbArtistIDs:
                dbArtistIDAlbums = mdb.getArtistAlbumsFromID(db, dbArtistID, flatten=True)
                                
                ma = matchAlbums()
                ma.match(knownArtistAlbums, dbArtistIDAlbums)
                if "ID" in debug or "Full" in debug:
                    print("\t\t{0: <45}{1}\t{2}\t{3}\t{4}".format(dbArtistID, len(dbArtistIDAlbums), ma.near, ma.score, ma.maxval))
                if ma.near < resultD["Matches"]:
                    continue
                if ma.score < max([resultD["Score"], cutoff]):
                    continue
                resultD = {"ID": dbArtistID, "Matches": ma.near, "Score": ma.score, "Best": ma}
                print("\t\t{0: <45}{1}\t{2}\t{3}\t{4} <-- Match".format(dbArtistID, len(artistAlbums), ma.near, ma.score, ma.maxval))

                
        if resultD["ID"] is not None:
            print("\t\t{0: <45}{1}\t{2} <====================================== Best Match".format(resultD["ID"], resultD["Matches"], resultD["Score"]))
            retval[db] = {'ID': resultD["ID"], 'Name': None}
            if "Full" in debug:
                print("\t\t =====>",retval[db])
        else:
            if "Full" in debug:
                print("\t\t =====> No Match")
            retval[db] = None
            
    return retval



def searchForMutualArtistDBEntries(mdb, artistName, num=2, cutoff=0.8, debug=[None]):
    retval     = {}

    
    ######################################################################
    #### Determine Albums To Match
    ######################################################################
    artistAlbums = []
    dbsToMatch   = []
    dbMatches    = mdb.getArtistDataIDs(artistName)
    knownDBs     = []
    for db,artistID in dbMatches.items():
        if artistID is not None:
            artistAlbums.append(mdb.getArtistAlbumsFromID(db, artistID, flatten=True))
            knownDBs.append(db)
        else:
            dbsToMatch.append(db)
    from listUtils import getFlatList
    knownArtistAlbums   = list(set(getFlatList(artistAlbums)))
    print("Searching for matches:  [{0}] using [{1}] albums collected from [{2}] dbs".format(artistName, len(knownArtistAlbums), len(artistAlbums)))
    print("  Will search for matches in these DBs: {0}".format(dbsToMatch))
    
    
    ######################################################################
    #### Loop Over Missing DBs
    ######################################################################
    retval = searchForArtistAlbumsDBEntries(mdb, artistName, knownArtistAlbums, dbsToMatch, cutoff, num, debug)
    return retval
            
            
def searchForMutualDBEntries(mdb, num=2, cutoff=0.8, debug=[None], minI=-1, maxR=50):
    retval = {}
    nR = 0
    
    musicArtists = mdb.getArtists()
    for i, artistName in enumerate(musicArtists):
        if i <= minI:
            continue
        result = searchForMutualArtistDBEntries(mdb, artistName, num, cutoff, debug)
        for db,dbval in result.items():
            if dbval is not None:
                if retval.get(artistName) is None:
                    retval[artistName] = {}
                retval[artistName][db] = dbval
                nR += 1
                
        if nR > maxR:
            break
                
    print("Found {0} new artist matches after looping over {1} artists".format(len(retval), i))
    return retval